# NeiBot EventSub ローカルE2Eテスト

このノートブックは FastAPI サーバーを起動し、Twitch EventSub 通知を署名付きで送信して DB 反映を検証します。
- 事前に `pip install -r requirements.txt` を実施してください
- `venv/token.json` に Twitch/Discord 設定を用意してください（secret は署名で使用）
- Discord Bot は起動不要（API のみ使用）


In [1]:
import os; os.chdir('..') 
# サーバー起動（127.0.0.1:8000）
import os, threading, time
os.environ.setdefault('DEBUG', '1')
from bot.bot_client import app
import uvicorn

def _run():
    uvicorn.run(app, host='127.0.0.1', port=8000, log_level='info')

# 既に他プロセスで動いている場合はこのセルの再実行は不要です
t = threading.Thread(target=_run, daemon=True)
t.start()
time.sleep(1.0)
print('Started FastAPI on http://127.0.0.1:8000')


INFO:     Started server process [35820]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] 通常、各ソケット アドレスに対してプロトコル、ネットワーク アドレス、またはポートのどれか 1 つのみを使用できます。
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Started FastAPI on http://127.0.0.1:8000


In [2]:
# 署名付き EventSub 通知送信ユーティリティ
import hmac, hashlib, uuid, json, time, requests

def _now_iso():
    return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

def _sign(secret: str, msg_id: str, msg_ts: str, body_bytes: bytes) -> str:
    mac = hmac.new(secret.encode('utf-8'), (msg_id + msg_ts).encode('utf-8') + body_bytes, hashlib.sha256)
    return 'sha256=' + mac.hexdigest()

def post_verification(challenge: str = 'hello', base='http://127.0.0.1:8000/twitch_eventsub'):
    body = {'challenge': challenge}
    b = json.dumps(body, ensure_ascii=False).encode('utf-8')
    headers = {
        'Twitch-Eventsub-Message-Id': str(uuid.uuid4()),
        'Twitch-Eventsub-Message-Timestamp': _now_iso(),
        'Twitch-Eventsub-Message-Type': 'webhook_callback_verification',
        'Content-Type': 'application/json',
    }
    r = requests.post(base, data=b, headers=headers, timeout=10)
    return r.status_code, r.text

def post_event(sub_type: str, event: dict, secret: str, base='http://127.0.0.1:8000/twitch_eventsub', version='1'):
    body = {
        'subscription': {'type': sub_type, 'version': version},
        'event': event,
    }
    b = json.dumps(body, ensure_ascii=False).encode('utf-8')
    msg_id = str(uuid.uuid4())
    msg_ts = _now_iso()
    headers = {
        'Twitch-Eventsub-Message-Id': msg_id,
        'Twitch-Eventsub-Message-Timestamp': msg_ts,
        'Twitch-Eventsub-Message-Type': 'notification',
        'Twitch-Eventsub-Message-Signature': _sign(secret, msg_id, msg_ts, b),
        'Content-Type': 'application/json',
    }
    r = requests.post(base, data=b, headers=headers, timeout=10)
    try:
        return r.status_code, r.json()
    except Exception:
        return r.status_code, r.text


In [3]:
# テスト用リンク（Discord<->Twitch ID 紐付け）を作成
from bot.utils.save_and_load import patch_linked_user, get_linked_user

DISCORD_ID = '999999999999999999'   # 任意のテスト用 Discord ID
TWITCH_UID = '114514'            # 任意のテスト用 Twitch user_id

patch_linked_user(DISCORD_ID, {
    'twitch_user_id': TWITCH_UID,
    'twitch_username': 'test_user',
    'is_subscriber': False,
    'tier': None,
})
get_linked_user(DISCORD_ID)


{'twitch_user_id': '114514',
 'twitch_username': 'test_user',
 'is_subscriber': False,
 'tier': '2000',
 'subscribed_since': '2025-09-07',
 'last_verified_at': '2025-09-17',
 'cumulative_months': 7,
 'streak_months': 4}

In [4]:
# secret 読み込み
from bot.utils.save_and_load import get_eventsub_config
cb, secret = get_eventsub_config()
print('Callback =', cb)
print('Secret   =', ('*' * len(secret)) if secret else '(missing)')


Callback = https://ed93b9f6929f.ngrok-free.app/twitch_eventsub
Secret   = ******************************


In [5]:
# 1) webhook 検証
post_verification('challenge-ok')


(200, 'challenge-ok')

In [6]:
# 2) subscribe 通知
status, res = post_event('channel.subscribe', {
    'user_id': TWITCH_UID,
    'tier': '1000',
}, secret)
print(status, res)
get_linked_user(DISCORD_ID)


200 {'status': 'ok', 'matched': 1}


{'twitch_user_id': '114514',
 'twitch_username': 'test_user',
 'is_subscriber': True,
 'tier': '1000',
 'subscribed_since': '2025-09-07',
 'last_verified_at': '2025-09-17',
 'cumulative_months': 7,
 'streak_months': 4}

In [7]:
# 3) subscription.message（再サブ）
status, res = post_event('channel.subscription.message', {
    'user_id': TWITCH_UID,
    'tier': '2000',
    'cumulative_months': 7,
    'streak_months': {'months': 4},  # dict でも int でも可
}, secret)
print(status, res)
get_linked_user(DISCORD_ID)


200 {'status': 'ok', 'matched': 1}


{'twitch_user_id': '114514',
 'twitch_username': 'test_user',
 'is_subscriber': True,
 'tier': '2000',
 'subscribed_since': '2025-09-07',
 'last_verified_at': '2025-09-17',
 'cumulative_months': 7,
 'streak_months': 4}

In [8]:
# 4) subscription.end（終了）
status, res = post_event('channel.subscription.end', {
    'user_id': TWITCH_UID,
}, secret)
print(status, res)
get_linked_user(DISCORD_ID)


200 {'status': 'ok', 'matched': 1}


{'twitch_user_id': '114514',
 'twitch_username': 'test_user',
 'is_subscriber': False,
 'tier': '2000',
 'subscribed_since': '2025-09-07',
 'last_verified_at': '2025-09-17',
 'cumulative_months': 7,
 'streak_months': 4}

In [9]:
# 受信履歴（webhook_events）を確認
import sqlite3, os
db = os.path.join(os.getcwd(), 'db.sqlite3')
con = sqlite3.connect(db)
cur = con.cursor()
cur.execute("SELECT source, delivery_id, event_type, status, received_at FROM webhook_events ORDER BY received_at DESC LIMIT 10")
rows = cur.fetchall()
con.close()
rows


[('twitch',
  'c429897c-72f8-4f65-9571-d8e1e2f0480d',
  'channel.subscription.end',
  'done',
  '2025-09-17T05:37:18.360068+00:00'),
 ('twitch',
  'fa303c24-d4f9-4789-9c2a-6a7d5aa1daea',
  'channel.subscription.message',
  'done',
  '2025-09-17T05:37:18.324734+00:00'),
 ('twitch',
  'f92e6c39-60f8-4282-aa1d-814c8d6be02f',
  'channel.subscribe',
  'done',
  '2025-09-17T05:37:18.287397+00:00'),
 ('twitch',
  '54cd08cb-8d68-4e6c-b516-ed4f643deb64',
  'channel.subscription.end',
  'done',
  '2025-09-17T04:39:21.586173+00:00'),
 ('twitch',
  'd74f575e-9142-4666-8209-9c4481fc79da',
  'channel.subscription.message',
  'done',
  '2025-09-17T04:39:20.999551+00:00'),
 ('twitch',
  'd9334904-b427-44bc-8487-0a197de19e73',
  'channel.subscribe',
  'done',
  '2025-09-17T04:39:16.565097+00:00'),
 ('twitch',
  'be0b8f67-1801-4b3d-bf93-d79a0a362131',
  'channel.subscription.end',
  'done',
  '2025-09-07T22:52:27.517174+00:00'),
 ('twitch',
  '3a1635fe-001e-437b-b1b3-bff2f97b751c',
  'channel.subscripti